<a href="https://colab.research.google.com/github/JuanJoseMV/neuraltextgen/blob/main/RNN_TextGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalations

###### Apex

In [2]:
%%capture
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
%%writefile setup.sh
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
# Writing setup.sh

Writing setup.sh


In [4]:
%%capture
!sh setup.sh

# Imports

In [5]:
import gensim.models.wrappers.fasttext
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
# from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModel, BertConfig, AutoConfig
from collections import Counter

# Cleaning the dataset


In [ ]:
## When using Wiki.tokens (not wiki.5k)

with open('/content/wiki.train.tokens') as f:
  content = f.readlines()

clean = []
for c in content:
  clean.append(c.replace('\n', '[EOS]'))

In [6]:
with open('/content/neuraltextgen/data/wiki103.5k.txt') as f:
  content = f.readlines()

clean = []
for c in content:
  clean.append(c.replace('@@UNKNOWN@@', ''))

In [7]:
! touch '/content/neuraltextgen/data/cleaned_wiki103.5k.txt'

In [8]:
with open('/content/neuraltextgen/data/cleaned_wiki103.5k.txt', 'w') as f:
    for item in clean:
        f.write("%s\n" % item)

# Downloading pre-trained wordembeddings

In [16]:
# It takes some minutes, avoid if won't use
model = gensim.models.KeyedVectors.load_word2vec_format('/content/wiki-news-300d-1M.vec')
word_vectors = model.wv

weights = torch.FloatTensor(word_vectors.vectors)
embedding = nn.Embedding.from_pretrained(weights)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


# Train the network

In [23]:
os.chdir('/content/neuraltextgen/')
from RNNGenerator import RNNGenerator

params = {
    "seq_size": 512, 
    "batch_size": 32, 
    "embedding_size": 300, 
    "lstm_size": 128,
    "lstm_num_layers": 3, 
    "lstm_bidirectional": True, 
    "lstm_dropout": 0.5, 
    "gradients_norm": 5,
    "predict_top_k": 10, 
    "training_epocs": 1000, 
    "lr": 0.01, 
    "weights": None
}

train_file = 'data/cleaned_wiki103.5k.txt'
generator = RNNGenerator(**params)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_net = generator.train(device, train_file)

# list of sentences
sentences = generator.predict(device, trained_net, n_sentences=100)
sentences

Vocabulary size 14104
Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic


RuntimeError: ignored